# Advanced NLP HW0 Reference

Build a text generator based on n-gram language model and neural language model.

    Find a corpus (e.g. http://www.lib.ru/SHAKESPEARE/hamlet2.txt), but you are free to use anything else of your interest
    Preprocess it if necessary (we suggest using nltk for that)
    Build an n-gram model
    Try out different values of n, calculate perplexity on a held-out set
    Build a simple neural network model for text generation (start from a feed-forward net for example). We suggest using tensorflow + keras for this task

Criteria:

    Data is split into train / validation / test, motivation for the split method is given
    N-gram model is implemented
    Neural network for text generation is implemented
    Perplexity is calculated for both models
    Examples of texts generated with different models are present and compared


In [1]:
import requests
import re
import string
from sklearn.model_selection import train_test_split
from collections import defaultdict
from numpy.random import choice
import numpy as np
from collections import Counter
from sklearn.model_selection import ParameterGrid
import pandas as pd

In [2]:
from tensorflow.keras.layers import Dense, Input, Embedding, Flatten, CuDNNLSTM
from tensorflow.keras import Model
from tensorflow.keras.callbacks import Callback

## Data preprocessing

In [3]:
corpus_url = 'http://lib.ru/POEZIQ/DANTE/comedy.txt'

In [4]:
response = requests.get(corpus_url)
raw_corpus = response.text

raw_corpus[:25000]

'<html><head><title>Данте Алигьери. Божественная комедия</title></head><body><pre><div align=right><form action=/POEZIQ/DANTE/comedy.txt><select name=format><OPTION VALUE="_Contents">Содержание<OPTION VALUE="_with-big-pictures.html">Fine HTML<OPTION VALUE="_with-big-pictures.html">Printed version<OPTION VALUE="_Ascii.txt">txt(Word,КПК)<OPTION VALUE="">Lib.ru html</select><input type=submit value=go></form></div><pre>\n<ul><a name=0></a><h2>Данте Алигьери. Божественная комедия</h2></ul>\n\n----------------------------------------------------------------------------\n     Перевод М.Лозинского\n     ББК 84.4 Ит\n         Д 17\n     Издательство "Правда", М.: 1982\n     OCR Бычков М.Н.\n----------------------------------------------------------------------------\n\n     "Божественная Комедия" возникла в тревожные ранние  годы  XIV  века  из\nбурливших напряженной политической борьбой глубин национальной жизни Италии.\nДля будущих  -  близких  и  далеких  -  поколений  она  осталась  велича

In [6]:
def preprocess(text):
    TAG_RE = re.compile(r'<[^>]+>') # removing tags
    corpus = TAG_RE.sub('', text) 
    first_chapter_index = corpus.find("АД") # looking for the start
    corpus = corpus[first_chapter_index:]
    remarks_index = corpus.find('ПРИМЕЧАНИЯ') # looking for the end
    corpus = corpus[:remarks_index]
    chapter_RE = re.compile(r'([А-Я]{2,}){1,3}') # removing chapter names
    corpus = re.sub(chapter_RE, ' ', corpus)  
    corpus = re.sub(r"\d+", "", corpus) # removing numbers 
    conversion_table = str.maketrans({key: '' for key in string.punctuation}) #removing punctuation
    corpus = corpus.translate(conversion_table) 
    corpus = re.sub(r' +', ' ', corpus) # substituting several whitespaces with a single one
    corpus = re.sub(r'(\n[ ]*)+', ' \n', corpus) # substituting patterns like \n \n \n  and \n\n\n with a single \n with leading whitespace
    corpus = corpus.lower() #lowercasing everything
    return corpus.strip() #stripping leading whitespaces

In [7]:
preprocessed_text = preprocess(raw_corpus)

In [8]:
preprocessed_text[:100]

'земную жизнь пройдя до половины \nя очутился в сумрачном лесу \nутратив правый путь во тьме долины \nка'

#### Train/val/test split
We will split text by lines.

More than a half of the text will be used as a training set.

The rest will be split between validaition and test.

In [9]:
line_split = preprocessed_text.split(' \n')
line_split[:10]

['земную жизнь пройдя до половины',
 'я очутился в сумрачном лесу',
 'утратив правый путь во тьме долины',
 'каков он был о как произнесу',
 'тот дикий лес дремучий и грозящий',
 'чей давний ужас в памяти несу',
 'так горек он что смерть едва ль не слаще',
 'но благо в нем обретши навсегда',
 'скажу про все что видел в этой чаще',
 'не помню сам как я вошел туда']

In [10]:
train_sentences, test_sentences = train_test_split(line_split, train_size = 0.6, shuffle=False)
val_sentences, test_sentences = train_test_split(test_sentences, train_size = 0.5, shuffle=False)

In [11]:
train_sentences[:10]

['земную жизнь пройдя до половины',
 'я очутился в сумрачном лесу',
 'утратив правый путь во тьме долины',
 'каков он был о как произнесу',
 'тот дикий лес дремучий и грозящий',
 'чей давний ужас в памяти несу',
 'так горек он что смерть едва ль не слаще',
 'но благо в нем обретши навсегда',
 'скажу про все что видел в этой чаще',
 'не помню сам как я вошел туда']

#### Fixing vocabulary

We will fix vocabulary for our models -- we will take 10000 most common tokens from the train sample.

In [12]:
cnt = Counter()
for line in train_sentences:
  cnt.update(line.split(' ') + ['\n'])

In [13]:
len(cnt)

14062

In [14]:
fixed_vocab = [v[0] for v in cnt.most_common(10000)]

In [15]:
fixed_vocab[:10]

['\n', 'и', 'в', 'я', 'не', 'как', 'он', 'что', 'на', 'так']

## Models

Base class for the model.

In [0]:
class BaseLM:
    def __init__(self, n, vocab=None):
        """Language model constructor
    n -- n-gram size
    vocab -- optional fixed vocabulary for the model
    """
        self.n = n
        self.vocab = vocab

    def prob(self, word, context=None):
        """This method returns probability of a word with given context: P(w_t | w_{t - 1}...w_{t - n + 1})
    
    For example:
    >>> lm.prob('hello', context=('world',))
    0.99988
    """
        raise NotImplementedError

    def generate_text(self, text_length):
        """This method generates random text of length 
    
    For example
    >>> lm.generate_text(2)
    hello world

    """
        raise NotImplementedError

    def update(self, sequence_of_tokens):
        """This method learns probabiities based on given sequence of tokents
    
    sequence_of_tokens -- iterable of tokens

    For example
    >>> lm.update(['hello', 'world'])
    """
        raise NotImplementedError

    def perplexity(self, sequence_of_tokens):
        """This method returns perplexity for a given sequence of tokens
    
    sequence_of_tokens -- iterable of tokens
    """
        raise NotImplementedError

## N-gram model

In [0]:
class NGramLM(BaseLM):
    def __init__(self,
                 n,
                 vocab=None,
                 smoothing_k=1,
                 unk='<UNK>',
                 left_pad='<s>',
                 right_pad='</s>'):
        """N-Gram model constructor
    n -- ngram size
    vocab -- optionally fixed vocabulary
    smoothing_k -- smoothing parameter
    unk -- substitute symbol for handling unknowns
    left_pad -- left padding symbol
    right_pad -- right padding symbol
    """
        self.smoothing_coef = smoothing_k
        super().__init__(n, vocab)
        self.count_context = defaultdict(lambda: defaultdict(lambda: 0))
        self.model = defaultdict(lambda: defaultdict(lambda: 0))
        self.learn_vocab = self.vocab is None
        self.vocab = vocab if vocab is not None else []
        self.left_pad = left_pad
        self.right_pad = right_pad
        self.unk = unk
        self.vocab = [self.left_pad, self.unk, self.right_pad] + self.vocab

    def update(self, sequence_of_tokens):
        """This method learns probabiities based on given sequence of tokents
    """

        #Padding to handle start tokens
        sequence_of_tokens_local = [self.left_pad] * (self.n - 1) + list(
            sequence_of_tokens) + [self.right_pad]

        #Substitution of unknown symbols
        if not self.learn_vocab:
            sequence_of_tokens_local = [
                s if s in self.vocab else self.unk for s in sequence_of_tokens
            ]

        #Generation of n-grams
        ngrams = zip(*[sequence_of_tokens_local[i:] for i in range(self.n)])

        #Learning vocab if necessary
        for ngram in ngrams:
            if self.learn_vocab:
                for v in ngram:
                    if v not in self.vocab:
                        self.vocab.append(v)

            #Context / word
            context = tuple(ngram[:-1])
            word = ngram[-1]

            #Update counts
            self.count_context[context][word] += 1

        #Setting smoothed probability if context is unknown
        self.model = defaultdict(lambda: 1 / len(self.vocab))

        for context in self.count_context.keys():
            #Denominator of k-smoothed P(w | w_i...w_{i-n+1}) MLE
            denominator = sum(self.count_context[context].values()
                              ) + self.smoothing_coef * len(self.vocab)

            #Setting smoothed probability if token is unknown
            self.model[context] = defaultdict(
                lambda: self.smoothing_coef / denominator)
            for word in self.count_context[context].keys():
                #Smothed probability estimate for known ngrams
                self.model[context][word] = (
                    self.smoothing_coef +
                    self.count_context[context][word]) / denominator

    def prob(self, word, context=None):
        """This method returns probability of a word with given context: P(w_t | w_{t - 1}...w_{t - n + 1})
    """

        #Try to find context
        context_probs = self.model[context]

        #Context is not found and the default probability returned
        if not isinstance(context_probs, defaultdict):
            return context_probs

        #Return probability estimate
        return self.model[context][word]

    def generate_text(self, text_length, sep=' '):
        """This method generates random text of length text_length
    """
        #Creaing starting context
        context = tuple([self.left_pad] * (self.n - 1))

        generated_tokens = []
        for t in range(text_length):
            #Distribution estimate over vocabulary
            distribution_over_vocab = [
                self.prob(v, context) for v in self.vocab
            ]

            #Generation of next token
            next_word = choice(self.vocab,
                               replace=False,
                               size=1,
                               p=np.array(distribution_over_vocab) /
                               sum(distribution_over_vocab))[0]
            generated_tokens.append(next_word)

            #Generation of a new context
            context = tuple(list(context[1:]) + [next_word])

        return sep.join(generated_tokens)

    def perplexity(self, sequence_of_tokens):
        """This method returns perplexity for a given sequence of tokens
    """
        #Padding sequence of tokens
        sequence_of_tokens_local = [self.left_pad] * (self.n - 1) + [
            s if s in self.vocab else self.unk for s in sequence_of_tokens
        ] + [self.right_pad]

        #Generation of n-grams
        ngrams = zip(*[sequence_of_tokens_local[i:] for i in range(self.n)])

        probs = []

        #Probability for each token
        for ngram in ngrams:
            context = tuple(ngram[:-1])
            word = ngram[-1]
            probs.append(np.log2(self.prob(word, context)))

        #Perplexity
        return 2**(-sum(probs) / len(sequence_of_tokens_local))

In [0]:
def lm_tuning(lm_class, params_dict, train_set, validation_set, verbose=0):
    """Method for tuning language model
    lm_class -- language model class
    params_dict -- dictionary of parameters for the model
    train_set -- sequence of tokens for training 
    validation_set -- sequence of tokens for validation
    verbose -- verbosity parameter 0 -- print nothing, or >0 -- prints scores and parameters
    """
    grid = ParameterGrid(params_dict)
    best_params = None
    best_perplexity = np.Inf
    best_model = None

    for param_set in grid:

    if verbose > 0:
        print(param_set)

    ngram = lm_class(**param_set)
    ngram.update(train_set)
    perp = ngram.perplexity(validation_set)

    if verbose > 0:
        print(perp)
    if np.less(perp, best_perplexity):
        best_params = param_set
        best_perplexity = perp
        best_model = ngram

    if verbose > 0:
    print('Best params: {}\nBest perplexity: {}'.format(best_params, best_perplexity))
    return best_model

In [18]:
' \n '.join(train_sentences[:5]).split(' ')

['земную',
 'жизнь',
 'пройдя',
 'до',
 'половины',
 '\n',
 'я',
 'очутился',
 'в',
 'сумрачном',
 'лесу',
 '\n',
 'утратив',
 'правый',
 'путь',
 'во',
 'тьме',
 'долины',
 '\n',
 'каков',
 'он',
 'был',
 'о',
 'как',
 'произнесу',
 '\n',
 'тот',
 'дикий',
 'лес',
 'дремучий',
 'и',
 'грозящий']

In [0]:
ngram = lm_tuning(NGramLM,
                  params_dict = {'n': list(range(2,6)),
                                'smoothing_k': [0.001, 0.01, 0.1, 0.5, 1.0],
                                'vocab':[fixed_vocab]},
                  train_set=' \n '.join(train_sentences).split(' '),
                  validation_set=' \n '.join(val_sentences).split(' '),
                  verbose=1)

{'n': 2, 'smoothing_k': 0.001, 'vocab': ['\n', 'и', 'в', 'я', 'не', 'как', 'он', 'что', 'на', 'так', 'к', 'с', 'ты', '', 'мне', 'был', 'но', 'кто', 'а', 'мой', 'где', 'их', 'когда', 'мы', 'о', 'от', 'бы', 'там', 'все', 'тот', 'здесь', 'его', 'из', 'за', 'сказал', 'чтоб', 'меня', 'то', 'по', 'у', 'чем', 'нам', 'нас', 'если', 'вот', 'это', 'же', 'этот', 'вождь', 'уже', 'ни', 'ему', 'до', 'для', 'они', 'над', 'всех', 'тебя', 'ним', 'который', 'раз', 'тебе', 'молвил', 'чтобы', 'ее', 'под', 'сам', 'тут', 'путь', 'она', 'во', 'те', 'лишь', 'один', 'дух', 'пока', 'свой', 'без', 'им', 'тех', 'нет', 'еще', 'потом', 'только', 'учитель', 'твой', 'да', 'мной', 'речь', 'вы', 'видел', 'тем', 'взгляд', 'было', 'была', 'со', 'взор', 'есть', 'иль', 'вас', 'меж', 'быть', 'свет', 'начал', 'ли', 'ей', 'ответ', 'или', 'них', 'ответил', 'мог', 'нем', 'этой', 'тогда', 'сквозь', 'себя', 'вновь', 'этих', 'такой', 'стал', 'теперь', 'хоть', 'пред', 'эти', 'может', 'нами', 'себе', 'ней', 'были', 'вдруг', 'затем',

In [0]:
ngram.perplexity(' \n '.join(test_sentences).split(' '))

235.64339119774746

In [0]:
print(ngram.generate_text(1000))

вредоносной смысла комунибудь дремучий рассталась капаней в желчь родимым гребнем перевидал выстрел груда скал обагривших кровью утомленный опустился преклонись во лес разгородили мертвыми падай трепещущее швырять скорбный склеп теченью давившей достойные мненье верного вождя уста голени ли кто размышлял до тронь презренье головными впереди неутомимо томный мечет их всех пособя найди борьбе могло поясом нашем истину полмили растения бездонный шьют будто вера уроки убитый 
 пылью рубежа окраска житья смертью осквернено плечи только мне монтаперти острые лежавших это высоте втайне казнишь пятки горя 
 иссох светит ведущей души два брату если судит войны гнусный огибая арахна пройти исступленный молится кумиру 
 коз объята вековечной нежданно грянул сомнением цепь докончив вздохи куколь тьму сильней огни 
 я речь вести гадины меняют язвит своевольно достойные величавой 
 распаленный призывать сужу пред именем твоим носящий хрипа скопища пей ручьи края властелин привстал умерщвленной измла

## Neural language model

In [0]:
class PerplexityEarlyStopping(Callback):
  """Callback for early stopping based on perplexity on the validation set
  """
  def __init__(self, validation_data):
    super().__init__()
    self.contexts, self.targets = validation_data
    self.best = np.Inf
  
  def on_epoch_end(self, epoch, logs=None):
    pred = self.model.predict(self.contexts)
    probs = [np.log2(pred[i, target[0]]) for i, target in enumerate(self.targets)]
    perp = 2**(-sum(probs) / self.targets.shape[0])
    print('- perplexity: {}'.format(perp))
    if np.less(perp, self.best):
      self.best = perp
    else:
      self.model.stop_training = True
      print('Early stopping due to higher perplexity')


In [0]:
class NeuralLM(BaseLM):
    def __init__(self,
                 n,
                 vocab=None,
                 unk='<UNK>',
                 left_pad='<s>',
                 right_pad='</s>'):
        """Neural language model constructor
    n -- ngram size
    vocab -- optionally fixed vocabulary
    unk -- substitute symbol for handling unknowns
    left_pad -- left padding symbol
    right_pad -- right padding symbol
    """

        super().__init__(n, vocab)
        self.learn_vocab = vocab is None
        self.vocab = vocab if vocab is not None else []
        self.left_pad = left_pad
        self.right_pad = right_pad
        self.unk = unk
        self.vocab = [self.left_pad, self.unk, self.right_pad] + self.vocab
        self.word_idx = {v: i for i, v in enumerate(self.vocab)}
        self.model = None

    def build_model(self):
        """Method for building neural net
    """
        inp = Input(shape=self.n - 1, name='sequence_input')
        emb = Embedding(input_dim=len(self.vocab),
                        output_dim=50,
                        input_length=self.n - 1)(inp)
        flat = Flatten()(emb)
        dense = Dense(len(self.vocab), activation='softmax')(flat)
        model = Model(inputs=[inp], outputs=[dense])
        model.compile(optimizer='adam',
                      loss='sparse_categorical_crossentropy',
                      metrics=['accuracy'])
        return model

    def update(self, sequence_of_tokens, validation_sequence=None):
        """This method learns probabiities based on given sequence of tokents
    """

        #Padding given sequence
        sequence_of_tokens_local = [self.left_pad] * (self.n - 1) + list(
            sequence_of_tokens) + [self.right_pad]

        #Substituting unknown tokens
        if not self.learn_vocab:
            sequence_of_tokens_local = [
                s if s in self.vocab else self.unk
                for s in sequence_of_tokens_local
            ]

        #Generation of n-grams
        ngrams = zip(*[sequence_of_tokens_local[i:] for i in range(self.n)])

        contexts = []
        targets = []

        #Learing vocab if necessary
        #Forming train matrix and lebels vector
        for ngram in ngrams:
            if self.learn_vocab:
                for v in ngram:
                    if v not in self.vocab:
                        self.word_idx[v] = len(self.vocab)
                        self.vocab.append(v)

            context = [self.word_idx[v] for v in ngram[:-1]]
            contexts.append(context)

            target = [self.word_idx[ngram[-1]]]
            targets.append(target)

        self.model = self.build_model()

        contexts_val = []
        targets_val = []

        #If validation data is given -- generate matrix and targets vector
        if validation_sequence is not None:
            validation_sequence_local = [self.left_pad] * (self.n - 1) + [
                s if s in self.vocab else self.unk for s in validation_sequence
            ] + [self.right_pad]
            ngrams_val = zip(
                *[validation_sequence_local[i:] for i in range(self.n)])

            for ngram in ngrams_val:
                context = [self.word_idx[v] for v in ngram[:-1]]
                contexts_val.append(context)
                target = [self.word_idx[ngram[-1]]]
                targets_val.append(target)

        validation_data = [np.array(contexts_val),
                           np.array(targets_val)
                           ] if validation_sequence is not None else None

        #Early stopping based on perplexity on validation set
        callbacks = None if validation_data is None else [
            PerplexityEarlyStopping(validation_data=validation_data)
        ]

        self.model.fit(x=np.array(contexts),
                       y=np.array(targets),
                       batch_size=256,
                       epochs=100,
                       validation_data=validation_data,
                       callbacks=callbacks)

    def prob(self, word, context=None):
        """This method returns probability of a word with given context: P(w_t | w_{t - 1}...w_{t - n + 1})
    """

        input_seq = np.array([[self.word_idx[v] for v in context]])
        pred = self.model.predict(input_seq)
        return pred[0, self.word_idx[word]]

    def generate_text(self, text_length, sep=' '):
        """This method generates random text of length text_length
    """
        context = tuple([self.left_pad] * (self.n - 1))

        generated_tokens = []

        for t in range(text_length):
            #Estimate of the distribution over vocabulary
            input_seq = np.array([[self.word_idx[v] for v in context]])
            pred = self.model.predict(input_seq)
            distribution_over_vocab = [
                pred[0, self.word_idx[v]] for v in self.vocab
            ]

            #Generation of the next token
            next_word = choice(self.vocab,
                               replace=False,
                               size=1,
                               p=np.array(distribution_over_vocab) /
                               sum(distribution_over_vocab))[0]
            generated_tokens.append(next_word)

            #Updating context
            context = tuple(list(context[1:]) + [next_word])
        return sep.join(generated_tokens)

    def perplexity(self, sequence_of_tokens):
        """This method returns perplexity for a given sequence of tokens
    """

        #Padding
        sequence_of_tokens_local = [self.left_pad] * (self.n - 1) + [
            s if s in self.vocab else self.unk for s in sequence_of_tokens
        ] + [self.right_pad]

        #Generation of n-grams
        ngrams = zip(*[sequence_of_tokens_local[i:] for i in range(self.n)])

        probs = []

        #Probability calculation
        for ngram in ngrams:
            context = tuple(ngram[:-1])
            word = ngram[-1]
            probs.append(np.log2(self.prob(word, context)))

        return 2**(-sum(probs) / len(sequence_of_tokens_local))

In [0]:
nnlm = NeuralLM(n = 4, vocab=fixed_vocab)

In [0]:
nnlm.update(' \n '.join(train_sentences).split(' '), validation_sequence=' \n '.join(val_sentences).split(' '))

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Train on 57731 samples, validate on 18932 samples
Epoch 1/100
57731/57731 [==============================] - 4s 73us/sample - loss: 8.0795 - acc: 0.1015 - val_loss: 5.5904 - val_acc: 0.1842
Epoch 2/100
57731/57731 [==============================] - 4s 64us/sample - loss: 6.5936 - acc: 0.1742 - val_loss: 5.3304 - val_acc: 0.2175
Epoch 3/100
57731/57731 [==============================] - 4s 64us/sample - loss: 6.2982 - acc: 0.1869 - val_loss: 5.2033 - val_acc: 0.2498
Epoch 4/100
57731/57731 [==============================] - 4s 65us/sample - loss: 6.0830 - acc: 0.1954 - val_loss: 5.1354 - val_acc: 0.2604
Epoch 5/100
57731/57731 [==============================] - 4s 63us/sample - loss: 5.8764 - acc: 0.2021 - val_loss: 5.0744 - val_acc: 0.2732
Epoch 6/100
57

In [0]:
nnlm.perplexity(' \n '.join(test_sentences).split(' '))

151.68401143406666

In [0]:
print(nnlm.generate_text(1000))

племен 
 я так как бы мы из <UNK> и желанье 
 где огней раз знай напротив вскричал <UNK> увидеть бы чреда 
 ты то под мной напрасно 
 такой фьоренца башни пел так скрыться понемногу 
 и отдыха шестым были судя крикнул больше была вожатый 
 идем лает вонзился был как до <UNK> снова 
 что нежной даже внемлют ее облечен стремила 
 он что мы брата их взойдешь ему кому 
 уже с ним не сердится лета 
 скорбный прах всех попустому 
 звезде донимает ту отпрыск дом 
 когда нам пора ран я про грудь светлый впадин не нас свой <UNK> 
 зло <UNK> от сюда иль движенья 
 сказал все вновь начал с ним тело те за палицей странный посад осталась защита чутьчуть 
 он в твой мире столь сородичем слова р 
 из подо не отяжелел за ты бы <UNK> ни шел и <UNK> и фивян 
 рассеяв птицей светлым промолвить властелином затем 
 утратив слышали летит с нами 
 пред их реку 
 не <UNK> не <UNK> как были тоскливо среди обе можем некогда <UNK> 
 и все наступать она я был <UNK> 
 ты своем лицом 
 еще так его <UNK> и великий 


## Models comparison

Let's create several models with different context size.

We will compare perplexities and will look at generated texts.

In [0]:
df_comp_model = pd.DataFrame([], columns=['model_type', 'n', 'train_perplexity', 'val_perplexity','test_perplexity', 'generated text'])

Running n-gram models for n in [2, 10] and smoothing parameter chosen on validation set.

In [0]:
for i in range(2, 11):
  model_type = 'ngram_lm'
  n = i
  ngram = lm_tuning(NGramLM,
                    params_dict = {'n': [i],
                                   'smoothing_k': [0.001, 0.01, 0.1, 0.5, 1.0],
                                   'vocab':[fixed_vocab]},
                    train_set=' \n '.join(train_sentences).split(' '),
                    validation_set=' \n '.join(val_sentences).split(' '),
                    verbose=0)
  
  train_perplexity=ngram.perplexity(' \n '.join(train_sentences).split(' '))
  val_perplexity=ngram.perplexity(' \n '.join(val_sentences).split(' '))
  test_perplexity=ngram.perplexity(' \n '.join(test_sentences).split(' '))
  generated_text = ngram.generate_text(1000)
  df_comp_model = df_comp_model.append(pd.DataFrame([[model_type, n, train_perplexity, val_perplexity, test_perplexity, generated_text]],
                                    columns=['model_type', 'n', 'train_perplexity', 'val_perplexity','test_perplexity', 'generated text']),ignore_index=True)

Running Neural Language models for n in [2, 10] and early stopping by perplexity on validation set.

In [0]:
for i in range(2, 11):
  model_type = 'neural_lm'
  n = i
  nnlm = NeuralLM(n = n, vocab=fixed_vocab)
  nnlm.update(' \n '.join(train_sentences).split(' '),
              validation_sequence=' \n '.join(val_sentences).split(' '))
  
  train_perplexity=nnlm.perplexity(' \n '.join(train_sentences).split(' '))
  val_perplexity=nnlm.perplexity(' \n '.join(val_sentences).split(' '))
  test_perplexity=nnlm.perplexity(' \n '.join(test_sentences).split(' '))
  generated_text = nnlm.generate_text(1000)

  df_comp_model = df_comp_model.append(pd.DataFrame([[model_type, n, train_perplexity, val_perplexity, test_perplexity, generated_text]],
                                    columns=['model_type', 'n', 'train_perplexity', 'val_perplexity','test_perplexity', 'generated text']),ignore_index=True)

Train on 57731 samples, validate on 18932 samples
Epoch 1/100
57731/57731 [==============================] - 4s 62us/sample - loss: 8.7050 - acc: 0.1152 - val_loss: 6.9209 - val_acc: 0.2269
Epoch 2/100
57731/57731 [==============================] - 3s 57us/sample - loss: 7.3679 - acc: 0.1810 - val_loss: 6.0561 - val_acc: 0.2602
Epoch 3/100
57731/57731 [==============================] - 3s 59us/sample - loss: 6.9173 - acc: 0.1967 - val_loss: 5.7289 - val_acc: 0.2502
Epoch 4/100
57731/57731 [==============================] - 3s 58us/sample - loss: 6.5703 - acc: 0.1969 - val_loss: 5.4989 - val_acc: 0.2427
Epoch 5/100
57731/57731 [==============================] - 3s 58us/sample - loss: 6.2872 - acc: 0.1974 - val_loss: 5.3522 - val_acc: 0.2479
Epoch 6/100
57731/57731 [==============================] - 3s 58us/sample - loss: 6.0675 - acc: 0.1994 - val_loss: 5.2766 - val_acc: 0.2413
Epoch 7/100
57731/57731 [==============================] - 3s 58us/sample - loss: 5.8964 - acc: 0.2022 - val_l

In [0]:
df_comp_model.sort_values('test_perplexity')

,model_type,n,train_perplexity,val_perplexity,test_perplexity,generated text
13,neural_lm,6,89.605030,148.753776,149.990080,прахе милый сердцах он который слушать скаты в...
12,neural_lm,5,94.104479,153.765799,153.054194,упала \n когда взор по и взоры \n в душе связа...
14,neural_lm,7,95.567560,152.959634,153.652414,зренье судив б пытливо братьям \n мирской поск...
11,neural_lm,4,143.000692,156.247632,154.099211,током пришло которые нас медлил мошной не спол...
16,neural_lm,9,85.246875,156.179912,156.020558,имя мне над а я вновь латинской светлеет видиш...
15,neural_lm,8,191.063043,157.139328,156.401840,глазами челе \n речь сковал если но воскликнул...
17,neural_lm,10,64.616685,156.761408,157.124953,вершина одни которым \n на он и мне <UNK> \n и...
10,neural_lm,3,177.020553,159.281767,157.311289,тьме входящим ногам \n как его горит \n <UNK> ...
9,neural_lm,2,187.314650,174.799741,173.128088,заметив суставы пятым \n они предстали растил ...
0,ngram_lm,2,43.664290,244.084995,235.643391,пиза ускорим звуком однажды стычкам разговору ...


The table above shows that neural language model generally outperforms n-gram model in terms of perplexity.

Let us take a look at the generated texts.

### Text generated by the model with the lowest test perplexity

In [0]:
print(df_comp_model.sort_values('test_perplexity')['generated text'].values[0])

прахе милый сердцах он который слушать скаты вышине 
 поведай чьей беседы платками владыкам 
 так здесь промолвил в голове камилла 
 раз была кого медля <UNK> 
 так то что что немало ров 
 кто лишь он и веры добру чести фуччи 
 тут явно дыме нему как пестрого куда 
 я забияка ногой сейчас мешок 
 взмывал сам меру распознать вперед изогнул на стран 
 от сограждан он и я со в лад 
 чем он такой не и была 
 взглянув не ними когда скала 
 и собой на обличье виду  
 мне он <UNK> как долгих тело 
 так ты он и обуянный 
 найти моей земляк встал 
 калечит чудо я отца толпы напротив зияло 
 чему все что врагом его неизреченной 
 к нам и ступай им 
 что мой вождь почет межи 
 взгляни ваше таким кто жизнь повторит 
 но я бегущий взял светил же смотришь которым 
 я сказал пошел мой к другом 
 но другая <UNK> <UNK> красоту 
 когда эней сестра беатриче божий 
 тебе в <UNK> примиренья 
 чем начал нессу смыкаются мало 
 потом огней вы крылья нет 
 его некий и дальше положен 
 чтобы тот кто на так от <

### Text generated by the language model with lowest test perplexity

In [0]:
print(df_comp_model[df_comp_model['model_type'] == 'ngram_lm'].sort_values('test_perplexity')['generated text'].values[0])

пиза ускорим звуком однажды стычкам разговору блудной темной приводит улыбкой держись висячий прочно влипли странно верхним заключено даль глазами сдирающие хрюкала разумом планета мглистыми взирая 
 а душу в бранный которая арбию слева луч короткий побед ничьих положив отверстье платимый горестных искуплен кулака скупым флоринам чую адом размахнулся обрушен работой папа ждем  скрывалась став сидел родине прегражденный рассказом  
 обрывистый убедишься дивом оков никого хвалу щедрей 
 остановилось божьих темных печальные умножилась сихея вещуном башке вопросу делал остановился изумлен скользнуть 
 я прижать гауденты жившая платы прельщен камни воздуху изведал 
 так разит стихах мое желанье знать взгляд злое сокол золото признаем всеместный одиноких брали выбрав ведя медленной принимает малое предстали двое слонах ладанные простору иной вели потреба луни каменное земная любя встретит ветрил постриженный пришли чащей убийцы попрал корок уголино фонарь возбранив достойным видом объемлющим

Text generated with the network looks much better, but still not very coherent.